# 코드 실행기
`15_code_interpreter.ipynb`

In [10]:
from dotenv import load_dotenv

load_dotenv()

True

## State

In [13]:
from langgraph.graph import MessagesState
from typing_extensions import Any

class State(MessagesState):
    question: str  # 사용자 질문
    dataset: Any   # 임의이 데이터셋(추후엔 DB에서 가져오거나 해야함)
    code: str      # 파이썬 코드 블럭
    result: str    # `code`의 실행결과
    answer: str    # 최종 답변

## Node

In [ ]:
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict, Annotated

llm = ChatOpenAI(model='gpt-4o', temperature=0)

class CodeBlock(TypedDict):
    code: Annotated[str, ..., '바로 실행 가능한 파이썬 코드']

def generate_code(state: State):
    prompt = f'''
    사용자 질문과 데이터셋을 제공할거야. 사용자 질문에 답변하기 위한 파이썬 코드를 생성해 줘.
    코드는 간단할수록 좋고, numpy, pandas, scikit-learn, scipy 가 설치되어 있으니 편하게 사용해.
    [주의] 이 코드는 실행될거기 때문에, 위험한 코드는 작성하면 안돼!
    ---
    질문: {state['question']}
    ---
    데이터셋: {state['dataset']}
    ---
    코드: 
    '''
    s_llm = llm.with_structured_output(CodeBlock)
    res = s_llm.invoke(prompt)
    return {'code': res['code']}


def execute_code(state: State):
    pass


def generate_answer(state: State):
    pass

import numpy as np

data = [1, 2, 3, 4, 5]

# Calculate the mean of the dataset
mean_value = np.mean(data)

print(f"평균은 {mean_value}입니다.")
